This is the tutorial for the evaluation of CPHF alchemical derivatives using PySCF (version 2.2)  

In [6]:
from pyscf import gto,scf
import numpy as np
import pyscf
pyscf.__version__

'2.2.0'

In [20]:
from pyscf.hessian import rks as rks_hess

## Fractional charge molecules

In [21]:
from FcMole import FcM, FcM_like

In [22]:
mol_NN=gto.M(atom= "N 0 0 0; N 0 0 2.1",unit="Bohr", basis="sto-3g") 

In [23]:
mf=scf.RKS(mol_NN)
mf.xc="PBE0"
mf.scf()

converged SCF energy = -107.94391478573


-107.94391478573009

In [26]:
mf.Hessian().hess()

array([[[[-9.82065548e-02,  3.05717590e-16,  4.83565606e-15],
         [ 6.71299874e-16, -9.82065548e-02,  4.14700177e-15],
         [ 4.23365374e-15,  3.71536643e-15,  2.10536578e+00]],

        [[ 9.81076526e-02, -2.82553138e-17,  1.18704316e-16],
         [-2.70414287e-17,  9.81076526e-02, -2.11330808e-16],
         [ 1.42828321e-16, -2.13452470e-16, -2.10522050e+00]]],


       [[[ 9.81076526e-02, -2.70414287e-17,  1.42828321e-16],
         [-2.82553138e-17,  9.81076526e-02, -2.13452470e-16],
         [ 1.18704316e-16, -2.11330808e-16, -2.10522050e+00]],

        [[-9.82065548e-02, -2.87084243e-15, -2.11565114e-15],
         [-4.17781386e-15, -9.82065548e-02, -3.88847252e-15],
         [-2.25882607e-15, -2.97265420e-15,  2.10536578e+00]]]])

## Using the Alchemical Perturbator (AP) class
We created an Alchemical perturbator object to access easily all the CPHF methods for alchemical derivative and alchemical forces derivatives, this allow us to save the derivatives and also to exploit the symmetry of the system

In [11]:
%load_ext autoreload
%autoreload 2
from AP_class import APDFT_perturbator as AP

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: cannot import name 'cpks' from 'pyscf.scf' (/home/giorgio/anaconda3/lib/python3.8/site-packages/pyscf/scf/__init__.py)

The alchemical perturbator is instantiated from a converged RHF object, and for some given perturbation sites.

In [5]:
mf_nn=scf.RHF(mol_NN)
mf_nn.scf()
ap_nn=AP(mf_nn,sites=[0,1])

converged SCF energy = -107.49885049543


NameError: name 'AP' is not defined

Alchemical gradient $ \partial E/\partial Z_i $, hessian $\partial^2E/\partial Z_i\partial Z_j$ and cubic hessian $\partial^3E/\partial Z_i\partial Z_j\partial Z_k$can be obtained from their buid functions 


In [47]:
#build the alchemical gradient dE/dZ_i
ap_nn.build_gradient()

array([-17.96252176, -17.96252176])

In [48]:
#build the alchemical hessian d**2E/dZ_i/dZ_j
ap_nn.build_hessian()

array([[-0.56261427,  0.95186321],
       [ 0.95186321, -0.56261427]])

In [49]:
ap_nn.build_cubic_hessian()

array([[[0.05028813, 0.00297524],
        [0.00297524, 0.00297524]],

       [[0.00297524, 0.00297524],
        [0.00297524, 0.05028813]]])

Are saved inside the class and can be accessed in a later moment

In [50]:
ap_nn.gradient,ap_nn.hessian,ap_nn.cubic_hessian

(array([-17.96252176, -17.96252176]),
 array([[-0.56261427,  0.95186321],
        [ 0.95186321, -0.56261427]]),
 array([[[0.05028813, 0.00297524],
         [0.00297524, 0.00297524]],
 
        [[0.00297524, 0.00297524],
         [0.00297524, 0.05028813]]]))

Alchemical perturbations for isolelectronic transmutations can be calculated from the derivatives up to order 3

In [51]:
ap_nn.APDFT1(np.asarray([-1,0])) # to CN-

-89.53632873812916

In [52]:
ap_nn.APDFT2(np.asarray([-1,1])) # to CO

-109.01332797130455

In [53]:
ap_nn.APDFT3(np.asarray([0,1]))  # to NO+

-125.7342980336741

In [54]:
ap_nn.APDFT3(np.asarray([1,1])) # to OO++

-143.01490712587218

## Alchemical forces
Alchemical forces are calculated resusing the density matrix derivatives already evaluated. The function af(i) gives the alchemical force of the atom $i$ : $\partial \mathbf{g}/ \partial Z_i$ [2]


In [55]:
ap_nn.af(0),ap_nn.af(1)

(array([[ 4.97829873e-17, -2.41093253e-16,  1.62250798e-01],
        [-4.97829873e-17,  2.41093253e-16, -1.62250798e-01]]),
 array([[-9.38846548e-17, -5.65820343e-16,  1.62250798e-01],
        [ 9.38846548e-17,  5.65820343e-16, -1.62250798e-01]]))

## Basis set effects 
The class also include methods to calculate the energy of the target molecules, with its basis set and with the basis set of the reference,

In [56]:
ap_nn.target_energy([-1,1]),ap_nn.target_energy_ref_bs([-1,1])

converged SCF energy = -111.221914650137
converged SCF energy = -108.996149557208


(-111.22191465013725, -108.99614955720804)

The APDFT3 predictions approximate the energy of the molecule with the reference basis set 

In [57]:
ap_nn.APDFT3([-1,1])

-109.01332797130455

But we can correct it using the single atom basis set correction [1]

In [58]:
ap_nn.APDFT3([-1,1])+ap_nn.ap_bsec([-1,1])

-111.47291530309323

## References 

[1] Giorgio Domenichini, Guido Falk von Rudorff, and O. Anatole von Lilienfeld : "Effects of perturbation order and basis set on alchemical predictions", J. Chem. Phys. 153, 144118 (2020)

[2] Giorgio Domenichini, and O. Anatole von Lilienfeld: "Alchemical geometry relaxation", J. Chem. Phys. 156, 184801 (2022); https://doi.org/10.1063/5.0085817 